# Living Folding Graphing Blues Cipher
## Platonic Solids Cellular Automata with Pentatonic Sound

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, Audio
import ipywidgets as widgets
import mido
from pygame import time

# Initialize the graph
solids = {"Tetrahedron": "C", "Cube": "D", "Octahedron": "E", 
          "Dodecahedron": "G", "Icosahedron": "A"}
G = nx.Graph()
for solid, note in solids.items():
    G.add_node(solid, note=note, state=0)
    
G.add_edges_from([("Tetrahedron", "Cube"), ("Tetrahedron", "Octahedron"),
                 ("Cube", "Octahedron"), ("Cube", "Dodecahedron"),
                 ("Octahedron", "Icosahedron"), ("Dodecahedron", "Icosahedron")])

In [ ]:
# Cellular Automata Rules
def update_states(G):
    new_states = {}
    for node in G.nodes():
        neighbor_states = [G.nodes[n]['state'] for n in nx.neighbors(G, node)]
        # Rule: If majority of neighbors are active, flip state
        new_states[node] = 1 if sum(neighbor_states) > len(neighbor_states)/2 else 0
    
    for node in G.nodes():
        G.nodes[node]['state'] = new_states[node]
    return G

In [ ]:
# Interactive Visualization
from ipywidgets import interact

def visualize_graph(G, step):
    G = update_states(G)
    pos = nx.spring_layout(G, seed=42)
    
    plt.figure(figsize=(10,8))
    nx.draw(G, pos, 
            node_color=[G.nodes[n]['state'] for n in G.nodes()],
            cmap=plt.cm.Blues,
            with_labels=True)
    plt.show()
    
    # Generate and play sound for active nodes
    active_notes = [G.nodes[n]['note'] for n in G.nodes() if G.nodes[n]['state']]
    if active_notes:
        print(f"Playing notes: {', '.join(active_notes)}")
        # (Sound generation code would go here)

interact(visualize_graph, G=widgets.fixed(G), step=widgets.IntSlider(min=0, max=20, step=1))

In [ ]:
# MIDI Export Function
def export_to_midi(G, filename="platonic_blues.mid"):
    mid = mido.MidiFile()
    track = mido.MidiTrack()
    mid.tracks.append(track)
    
    note_map = {'C': 60, 'D': 62, 'E': 64, 'G': 67, 'A': 69}
    
    for node in G.nodes():
        if G.nodes[node]['state']:
            track.append(mido.Message('note_on', note=note_map[G.nodes[node]['note']], 
                                    velocity=64, time=32))
            track.append(mido.Message('note_off', note=note_map[G.nodes[node]['note']], 
                                    velocity=64, time=128))
    
    mid.save(filename)
    print(f"Exported to {filename}")

# Example usage:
# export_to_midi(G)

## GitHub Publishing Instructions
1. Initialize repository: `git init`
2. Add files: `git add .`
3. Commit: `git commit -m "Initial commit of Platonic Blues Automata"`
4. Create GitHub repo and push